## A tutorial to visually compare zero-shot and one-shot learning results

### Approach
- Directly generate a DOT file from one-shot learning example completed in previous tutorial
- Load the image containing zero-shot graph
- Visually compare the difference

### Step 1: Download libraries
- Make use you download necessary library and files.
- All downloaded and saved files can be located in the `content` folder if using google Colab

In [ ]:
# uncomment the commands to download libraries and files
#!pip install python-dotenv
#!pip install dspy-ai==2.4.17
#!pip install graphviz
#!wget https://raw.githubusercontent.com/frankwxu/digital-forensics-lab/main/AI4Forensics/CKIM2024/PhishingAttack/PhishingAttackScenarioDemo/conversation.txt

import dspy
import os
import openai
import json
from dotenv import load_dotenv
from graphviz import Source
from IPython.display import Image, display


In [ ]:
from google.colab import files
uploaded = files.upload()

### Step 2: Config DSPy with openAI
- You `MUST` have an openAI api key
- load an openAI api key from `openai_api_key.txt` file
- or, hard code your open api key

In [ ]:
def set_dspy():
    # ==============set openAI enviroment=========
    # Path to your API key file
    key_file_path = "openai_api_key.txt"

    # Load the API key from the file
    with open(key_file_path, "r") as file:
        openai_api_key = file.read().strip()

    # Set the API key as an environment variable
    os.environ["OPENAI_API_KEY"] = openai_api_key
    openai.api_key = os.environ["OPENAI_API_KEY"]
    turbo = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=2000, temperature=0.5)
    dspy.settings.configure(lm=turbo)
    return turbo
    # ==============end of set openAI enviroment=========


def set_dspy_hardcode_openai_key():
    os.environ["OPENAI_API_KEY"] = "sk-proj-yourapikeyhere"
    openai.api_key = os.environ["OPENAI_API_KEY"]
    turbo = dspy.OpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=2000)
    dspy.settings.configure(lm=turbo)
    return turbo


# provide `openai_api_key.txt` with your openAI api key
turbo = set_dspy()
# optionally, hard code your openAI api key at line 21
# turbo=set_dspy_hardcode_openai_key()

### Step 3: Load the cyber incident repot (e.g., conversation)

- the same conversation as previous tutorial

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 02_output_email_analysis.png to 02_output_email_analysis.png
Saving 04_output.json to 04_output.json


In [ ]:
def load_text_file(file_path):
    """
    Load a text file and return its contents as a string.

    Parameters:
    file_path (str): The path to the text file.

    Returns:
    str: The contents of the text file.
    """
    try:
        with open(file_path, "r") as file:
            contents = file.read()
        return contents
    except FileNotFoundError:
        return "File not found."
    except Exception as e:
        return f"An error occurred: {e}"

conversation1 = load_text_file("04_output_for_viz.json")
conversation2 = load_text_file("03_output_for_viz.json")
print(conversation1)
print(conversation2)

[
    {
        "type": "identity",
        "id": "identity--1cba2e3c-4bdb-4d0b-a87b-2d504ad5923f",
        "name": "OurCompany",
        "identity_class": "organization",
        "sectors": [
            "technology"
        ],
        "contact_information": "info@ourcompany.com"
    },
    {
        "type": "email-addr",
        "id": "email-addr--0c0d2094-df97-45a7-9e9c-223569a9e798",
        "value": "hr-dept@ourcompany-security.com"
    },
    {
        "type": "email-message",
        "id": "email-message--c79b6bde-4f4c-4b38-a8c8-fb82921d6b97",
        "is_multipart": false,
        "subject": "Urgent Benefits Package Update",
        "from_ref": "email-addr--0c0d2094-df97-45a7-9e9c-223569a9e798",
        "body": "Please click the link to review the changes to your benefits package."
    },
    {
        "type": "url",
        "id": "url--4c3b-4c4b-bb6c-ded6b2a4a567",
        "value": "http://phishing-link.com/login"
    },
    {
        "type": "user-account",
        "id": "use

### Step 4: Tell an LLM `WHAT` are the inputs/outputs by defining `dspy.Signature`

In [ ]:
class DotGenerator(dspy.Signature):
    """Generate a evidence knowledge graph based on a cyber incident expressed in Structured Threat Information Expression (STIX)."""

    question: str = dspy.InputField(
        desc="a cyber incident expressed in Structured Threat Information Expression with JSON format."
    )

    answer: str = dspy.OutputField(
        desc="a graph in a dot format. The nodes of the graph are evidence entities and the edges of the graph are the relationships between evidence entities in STIX. A dot format is primarily associated with Graphviz, a graph visualization software. For example, a dot should looks like: digraph incident_name {...}. Don't include `````` "
    )

### Step 5: Tell an LLM `HOW` to generate answer. The business logic is defined in the module `.dspy.Predict`

In [ ]:
# Important: Predict is better than ChainOfThough
def generate_answer_CoT(signature, text, output_file):
    generate_answer = dspy.Predict(signature)
    answer = generate_answer(question=text).answer  # here we use the module

    with open(output_file, "w") as dot_file:
        print(answer)
        dot_file.write(answer)
        return answer
    print(f"The evidence has been saved to the file {output_file}")

### Step 6: Actually call LLM to generate the graph in a `.DOT` file

In [ ]:
output_file = "05_output.dot"
dot_description = generate_answer_CoT(
    DotGenerator,
    conversation1,
    output_file,
)

digraph phishing_attack {
  "incident--7a2b252e-c3e5-4bc2-bc6f-cb917ecf7857" [label="Phishing Attack on OurCompany", type="incident", first_seen="2024-07-17T08:00:00Z", last_seen="2024-07-17T08:10:00Z", status="ongoing"]
  "identity--1cba2e3c-4bdb-4d0b-a87b-2d504ad5923f" [label="OurCompany", type="identity", identity_class="organization", sectors="technology", contact_information="info@ourcompany.com"]
  "email-addr--0c0d2094-df97-45a7-9e9c-223569a9e798" [label="hr-dept@ourcompany-security.com", type="email-addr"]
  "email-message--c79b6bde-4f4c-4b38-a8c8-fb82921d6b97" [label="Urgent Benefits Package Update", type="email-message", is_multipart="false"]
  "url--4c3b-4c4b-bb6c-ded6b2a4a567" [label="http://phishing-link.com/login", type="url"]
  "user-account--bd5631cf-2af6-4bba-bc92-37c60d020400" [label="Taylor", type="user-account", user_id="Taylor", account_login="taylor@ourcompany.com"]
  "observable--001" [label="hr-dept@ourcompany-security.com", type="observable", observable_type="e

In [ ]:
output_file = "06_output.dot"
dot_description = generate_answer_CoT(
    DotGenerator,
    conversation2,
    output_file,
)

digraph phishing_email {
    "Phishing Email" -> "192.168.10.45"
    "Phishing Email" -> "banksecure.com"
    "Phishing Email" -> "banksecure-verification.com"
    "Phishing Email" -> "AccountDetails.exe"
    "Phishing Email" -> "e99a18c428cb38d5f260853678922e03"
    "Phishing Email" -> "10:20 AM"
    "Phishing Email" -> "Cleared browser history and cache"
    "Phishing Email" -> "Ran full antivirus scan"
    "Phishing Email" -> "Provided browser history entries and cookies"
    "Phishing Email" -> "Analyzed URLs and checked for suspicious files"
    "Phishing Email" -> "Quarantined malware file"
    "Phishing Email" -> "Checked network logs for IP 192.168.10.45"
    "Phishing Email" -> "Reset passwords and enabled two-factor authentication"
}


### Step 7: Visualize one-shot learning results from a `.DOT` file

In [ ]:
# Load the .DOT file
dot_file_path = "05_output.dot"

with open(dot_file_path, "r") as file:
    dot_content = file.read()

# Create a Graphviz Source object and render it
dot = Source(dot_content)

# Render the graph and save it as a PNG file
output_file_path = "05_output_stix_oneshot"
dot.format = "png"
dot.render(output_file_path, cleanup=True)

# Display the saved PNG file path
print(f"Graph saved as: {output_file_path}.png")

# Display the graph in the Jupyter notebook
dot

In [ ]:
# Load the .DOT file
dot_file_path = "06_output.dot"

with open(dot_file_path, "r") as file:
    dot_content = file.read()

# Create a Graphviz Source object and render it
dot = Source(dot_content)

# Render the graph and save it as a PNG file
output_file_path = "05_output_stix_zeroshot"
dot.format = "png"
dot.render(output_file_path, cleanup=True)

# Display the saved PNG file path
print(f"Graph saved as: {output_file_path}.png")

# Display the graph in the Jupyter notebook
dot

### Step 8: Visualize zero-shot learning results from previous tutorial

In [ ]:


image_path1 = "05_output_stix_zeroshot.png"
image_path2 = "05_output_stix_oneshot.png"


display(Image(filename=image_path1))
display(Image(filename=image_path2))

# Summary
- One-shot learning results make more sense when analyzing digital forensic evidence
- STIX mainly focus on cyber incident formalization, not forensic evidence
    - although basic evidence types are included in its document
- [Digital Forensic Knowledge Graph (DFKG)](https://github.com/frankwxu/digital-forensics-lab/tree/main/STIX_for_digital_forensics) is an extension of STIX
    - Cyber Forensic Domain Objects and Cyber Forensic Observable Objects